In [143]:
import numpy as np
import pandas as pd
from pyinform import *

In [144]:
SectorReturns = pd.read_csv("NormalisedSectorReturns.csv", index_col=0)
print(SectorReturns.shape)
SectorReturns.head()

(5291, 10)


,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
DF = SectorReturns.dropna()
DF.head()

,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-02-25,0.598655,-0.362595,-1.813326,-0.608180,-0.314499,-1.131099,-1.320868,-2.456540,0.725513,-1.709158
1992-02-26,1.482999,-0.536391,-1.203702,0.689234,1.158635,-0.013335,-0.304987,-1.497377,1.245510,0.459472
1992-02-27,1.932779,0.151317,0.188633,-0.021304,0.493030,-0.020952,-0.525962,-1.382264,0.621771,0.212791
1992-02-28,2.264313,0.003429,0.687197,0.274232,0.662564,-0.456263,-0.332417,-1.703039,0.413180,-0.870169
1992-03-02,1.119821,1.108068,1.012923,0.143172,0.636739,-0.686971,-0.686194,-1.379845,0.883358,-1.242292


In [146]:
for i in DF.columns:
    ts = DF[i]
    print(i, min(ts), max(ts))
    print(pd.qcut(ts, 5))
    print(" # # # # # # # ")

Energy -5.864528040336879 6.492276683616787
Date
1992-02-25                  (-0.293, 0.659]
1992-02-26                   (0.659, 1.515]
1992-02-27                   (1.515, 6.492]
1992-02-28                   (1.515, 6.492]
1992-03-02                   (0.659, 1.515]
                          ...              
2012-12-24                  (-0.293, 0.659]
2012-12-26                  (-0.293, 0.659]
2012-12-27                 (-1.231, -0.293]
2012-12-28    (-5.8660000000000005, -1.231]
2012-12-31                  (-0.293, 0.659]
Name: Energy, Length: 5124, dtype: category
Categories (5, interval[float64, right]): [(-5.8660000000000005, -1.231] < (-1.231, -0.293] < (-0.293, 0.659] < (0.659, 1.515] < (1.515, 6.492]]
 # # # # # # # 
Materials -9.843866063490458 6.922449313534987
Date
1992-02-25    (-1.203, -0.164]
1992-02-26    (-1.203, -0.164]
1992-02-27      (-0.164, 0.76]
1992-02-28      (-0.164, 0.76]
1992-03-02       (0.76, 1.543]
                    ...       
2012-12-24       (0.76, 

In [147]:
ts

Date
1992-02-25   -1.709158
1992-02-26    0.459472
1992-02-27    0.212791
1992-02-28   -0.870169
1992-03-02   -1.242292
                ...   
2012-12-24   -0.456415
2012-12-26   -1.302408
2012-12-27   -1.603746
2012-12-28   -3.201348
2012-12-31   -0.892309
Name: Health Care, Length: 5124, dtype: float64

In [148]:
pd.qcut(ts, 5, labels= range(5))

Date
1992-02-25    0
1992-02-26    2
1992-02-27    2
1992-02-28    1
1992-03-02    0
             ..
2012-12-24    1
2012-12-26    0
2012-12-27    0
2012-12-28    0
2012-12-31    1
Name: Health Care, Length: 5124, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

# Get Quantile-Binning

In [149]:
q = 100
QDF = pd.DataFrame()

for i in DF.columns:
    ts = DF[i]
    QDF[i] = pd.qcut(ts, q, labels=range(q))

def QuantileBin(DF, q):
    out = pd.DataFrame()

    for i in DF.columns:
        x = DF[i]
        out[i] = pd.qcut(x, q, labels = range(q))
    return(out)

QDF = QuantileBin(DF, 100)

In [150]:
QDF

,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-02-25,58,35,9,28,35,18,18,3,56,10
1992-02-26,79,32,19,57,69,40,38,14,70,50
1992-02-27,88,46,45,40,51,39,34,16,54,45
1992-02-28,92,43,58,46,56,31,38,11,49,23
1992-03-02,71,68,66,43,55,26,30,16,60,16
...,...,...,...,...,...,...,...,...,...,...
2012-12-24,42,76,64,6,32,42,13,54,59,32
2012-12-26,40,75,51,0,6,12,15,42,50,15
2012-12-27,34,63,44,4,18,11,17,36,46,11


# Transfer Entropy

Via package pyinform in https://www.researchgate.net/publication/323135385_Inform_A_toolkit_for_information-theoretic_analysis_of_complex_systems

cite at
https://ieeexplore.ieee.org/document/8285197

Documentation:
https://elife-asu.github.io/PyInform/_modules/pyinform/transferentropy.html

In [151]:
m = QDF.shape[1]
Causality = np.zeros((m,m))

for i in range(m):
    for j in range(m):
        I, J = QDF.columns[i], QDF.columns[j]
        Cij = transfer_entropy(source=QDF[I],
                            target = QDF[J], k=1)
        print(i,j,Cij)
        Causality[i,j] = Cij


def CausalTE(QDF, k=1):
    m = QDF.shape[1]
    Causality = np.zeros((m,m))

    for i in range(m):
        for j in range(m):
            I, J = QDF.columns[i], QDF.columns[j]
            Cij = transfer_entropy(source=QDF[I],
                            target = QDF[J], k=k)
            Causality[i,j] = Cij
    return(Causality)

0 0 0.0
0 1 4.274234589058012
0 2 4.351672776373434
0 3 4.466346194425987
0 4 4.325586709347523
0 5 4.3682831718731165
0 6 4.418221714309074
0 7 4.34236922195241
0 8 4.320203739143183
0 9 4.384748627905883
1 0 4.311581623601302
1 1 0.0
1 2 4.263197174479116
1 3 4.381083875663435
1 4 4.167382330048337
1 5 4.334758165829405
1 6 4.369080044960008
1 7 4.262446785921972
1 8 4.089265582660428
1 9 4.338400407254659
2 0 4.349761270962446
2 1 4.2252055142819325
2 2 0.0
2 3 4.330609781488786
2 4 4.144038445800519
2 5 4.325984212005895
2 6 4.346227825851218
2 7 4.298623861629563
2 8 4.127136172555445
2 9 4.271529360663465
3 0 4.399931678801401
3 1 4.280298142887288
3 2 4.265620684022458
3 3 0.0
3 4 4.222401886802309
3 5 4.324472511859819
3 6 4.358593460065823
3 7 4.339322387073094
3 8 4.227605699146251
3 9 4.235591389768162
4 0 4.339967279911437
4 1 4.150367500852426
4 2 4.168676195697374
4 3 4.303587369243001
4 4 0.0
4 5 4.336893896841335
4 6 4.366070282732973
4 7 4.190648495880405
4 8 4.0227225

In [152]:
Causality == CausalTE(QDF)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

In [153]:
Causaliy = CausalTE(QDF)

In [154]:
Causality - Causality.T

array([[ 0.        , -0.03734703,  0.00191151,  0.06641452, -0.01438057,
         0.01065496,  0.02222112,  0.00172461, -0.01177432,  0.01792198],
       [ 0.03734703,  0.        ,  0.03799166,  0.10078573,  0.01701483,
         0.04370764,  0.06034895,  0.04263687,  0.01626673,  0.05253624],
       [-0.00191151, -0.03799166,  0.        ,  0.0649891 , -0.02463775,
         0.00605471,  0.02391888, -0.00077631, -0.01505603,  0.01913365],
       [-0.06641452, -0.10078573, -0.0649891 ,  0.        , -0.08118548,
        -0.05746849, -0.04067983, -0.06069025, -0.07545606, -0.04399916],
       [ 0.01438057, -0.01701483,  0.02463775,  0.08118548,  0.        ,
         0.0236213 ,  0.04284803,  0.01644392, -0.00027387,  0.0381585 ],
       [-0.01065496, -0.04370764, -0.00605471,  0.05746849, -0.0236213 ,
         0.        ,  0.01454198, -0.01102968, -0.02506636,  0.00453425],
       [-0.02222112, -0.06034895, -0.02391888,  0.04067983, -0.04284803,
        -0.01454198,  0.        , -0.02649981

In [155]:
# Random Transfer Entropy
np.random.seed(1234)
print(QDF[I])
print(np.random.permutation(QDF[I].values))

Date
1992-02-25    10
1992-02-26    50
1992-02-27    45
1992-02-28    23
1992-03-02    16
              ..
2012-12-24    32
2012-12-26    15
2012-12-27    11
2012-12-28     1
2012-12-31    22
Name: Health Care, Length: 5124, dtype: category
Categories (100, int64): [0 < 1 < 2 < 3 ... 96 < 97 < 98 < 99]
[42 29 31 ... 90 75 97]


In [156]:
def Random(DF, seed = 1234):
    np.random.seed(seed)

    out = pd.DataFrame()

    for i in DF.columns:
        out[i] = np.random.permutation(DF[i].values)

    out.index = DF.index
    return(out)

In [157]:
Random(QDF)

,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-02-25,39,28,43,78,42,51,69,46,46,30
1992-02-26,24,47,7,38,81,38,85,20,13,11
1992-02-27,24,74,18,26,79,48,36,83,87,5
1992-02-28,41,32,92,29,11,72,22,45,69,4
1992-03-02,38,45,36,80,89,36,87,37,82,57
...,...,...,...,...,...,...,...,...,...,...
2012-12-24,80,7,61,45,85,3,76,94,33,14
2012-12-26,90,37,51,80,96,94,98,31,69,6
2012-12-27,85,72,27,88,12,88,60,79,47,63


In [158]:
CausalTE(Random(QuantileBin(DF, 100)))

array([[0.        , 4.78041983, 4.78043639, 4.79401349, 4.797439  ,
        4.76850729, 4.79064469, 4.79877185, 4.79657438, 4.79200985],
       [4.77663364, 0.        , 4.77986647, 4.78715743, 4.76871053,
        4.78029456, 4.79518468, 4.7844297 , 4.78573897, 4.8224125 ],
       [4.77538202, 4.77664631, 0.        , 4.7995671 , 4.78898237,
        4.77049062, 4.78750884, 4.79618139, 4.79772607, 4.81066585],
       [4.77615095, 4.76995792, 4.78441655, 0.        , 4.76806019,
        4.77409729, 4.77893962, 4.77730688, 4.77272545, 4.80343207],
       [4.79375354, 4.77076324, 4.79230326, 4.78379885, 0.        ,
        4.77961709, 4.79285157, 4.81728991, 4.78815755, 4.80657474],
       [4.77117913, 4.78714298, 4.77431287, 4.79619325, 4.78050884,
        0.        , 4.78651295, 4.8055509 , 4.78875623, 4.79374755],
       [4.77437086, 4.78503942, 4.77628938, 4.78599388, 4.78182479,
        4.77225203, 0.        , 4.7918396 , 4.7816529 , 4.7998813 ],
       [4.78188408, 4.77171851, 4.7827864

In [159]:
for i in range(4):
    s = np.random.randint(0,10000)
    CausalTE(Random(QuantileBin(DF, 100),s))
    print(s)

2278
339
7763
3047


In [160]:
def CompareRandom(DF, n_P, q):
    RTE = CausalTE(Random(QuantileBin(DF, q)))

    for i in range(1,n_P):
        s = np.random.randint(0,10000)
        print(s)
        RTE +=CausalTE(Random(QuantileBin(DF, q),s))

    return(RTE/float(n_P))

In [161]:
CompareRandom(DF,100,100).round(3)

2278
339
7763
3047
1913
8166
7460
8225
6695
2049
8635
8250
5459
7318
728
2203
6087
9641
4564
5637
2602
3312
6104
5423
9579
7205
2677
3406
5001
3757
96
1682
7395
7181
8293
131
4170
7907
8166
7460
8225
6695
2049
8635
8250
5459
7318
728
2203
6087
9641
4564
5637
2602
3312
6104
5423
9579
7205
2677
3406
5001
3757
96
1682
7395
7181
8293
131
4170
7907
8166
7460
8225
6695
2049
8635
8250
5459
7318
728
2203
6087
9641
4564
5637
2602
3312
6104
5423
9579
7205
2677
3406
5001
3757
96
1682
7395


array([[0.   , 4.786, 4.786, 4.788, 4.788, 4.782, 4.789, 4.789, 4.786,
        4.786],
       [4.782, 0.   , 4.787, 4.785, 4.788, 4.785, 4.786, 4.79 , 4.788,
        4.783],
       [4.781, 4.786, 0.   , 4.787, 4.792, 4.788, 4.789, 4.791, 4.788,
        4.784],
       [4.785, 4.786, 4.788, 0.   , 4.789, 4.786, 4.791, 4.79 , 4.785,
        4.785],
       [4.783, 4.787, 4.791, 4.786, 0.   , 4.787, 4.791, 4.789, 4.787,
        4.785],
       [4.78 , 4.788, 4.791, 4.788, 4.791, 0.   , 4.792, 4.79 , 4.785,
        4.782],
       [4.783, 4.784, 4.788, 4.789, 4.79 , 4.788, 0.   , 4.789, 4.788,
        4.783],
       [4.783, 4.789, 4.791, 4.788, 4.789, 4.786, 4.79 , 0.   , 4.785,
        4.787],
       [4.784, 4.79 , 4.79 , 4.786, 4.79 , 4.784, 4.791, 4.788, 0.   ,
        4.784],
       [4.786, 4.787, 4.789, 4.788, 4.79 , 4.783, 4.789, 4.791, 4.785,
        0.   ]])

In [162]:
# Random Transfer Entropy
n_P = 100 # number of permutations
RTE = CausalTE(Random(QuantileBin(DF, 100)))

for i in range(1,n_P):
    s = np.random.randint(0,10000)
    RTE +=CausalTE(Random(QuantileBin(DF, 100),s))

RTE = RTE/float(n_P)

In [163]:
print(RTE.round(3))

[[0.    4.786 4.786 4.788 4.788 4.782 4.789 4.789 4.786 4.786]
 [4.782 0.    4.787 4.785 4.788 4.785 4.786 4.79  4.788 4.783]
 [4.781 4.786 0.    4.787 4.792 4.788 4.789 4.791 4.788 4.784]
 [4.785 4.786 4.788 0.    4.789 4.786 4.791 4.79  4.785 4.785]
 [4.783 4.787 4.791 4.786 0.    4.787 4.791 4.789 4.787 4.785]
 [4.78  4.788 4.791 4.788 4.791 0.    4.792 4.79  4.785 4.782]
 [4.783 4.784 4.788 4.789 4.79  4.788 0.    4.789 4.788 4.783]
 [4.783 4.789 4.791 4.788 4.789 4.786 4.79  0.    4.785 4.787]
 [4.784 4.79  4.79  4.786 4.79  4.784 4.791 4.788 0.    4.784]
 [4.786 4.787 4.789 4.788 4.79  4.783 4.789 4.791 4.785 0.   ]]


In [164]:
Causality

array([[0.        , 4.27423459, 4.35167278, 4.46634619, 4.32558671,
        4.36828317, 4.41822171, 4.34236922, 4.32020374, 4.38474863],
       [4.31158162, 0.        , 4.26319717, 4.38108388, 4.16738233,
        4.33475817, 4.36908004, 4.26244679, 4.08926558, 4.33840041],
       [4.34976127, 4.22520551, 0.        , 4.33060978, 4.14403845,
        4.32598421, 4.34622783, 4.29862386, 4.12713617, 4.27152936],
       [4.39993168, 4.28029814, 4.26562068, 0.        , 4.22240189,
        4.32447251, 4.35859346, 4.33932239, 4.2276057 , 4.23559139],
       [4.33996728, 4.1503675 , 4.1686762 , 4.30358737, 0.        ,
        4.3368939 , 4.36607028, 4.1906485 , 4.02272256, 4.26499295],
       [4.35762821, 4.29105053, 4.3199295 , 4.38194101, 4.3132726 ,
        0.        , 4.32261593, 4.35388728, 4.30294099, 4.34681518],
       [4.39600059, 4.30873109, 4.32230895, 4.39927329, 4.32322225,
        4.30807395, 0.        , 4.34010563, 4.31621254, 4.35676402],
       [4.34064461, 4.21980991, 4.2994001

In [165]:
RelevantTE = Causality - RTE

In [166]:
RelevantTE.round(4)

array([[ 0.    , -0.5122, -0.4346, -0.322 , -0.4628, -0.4137, -0.3706,
        -0.4463, -0.4661, -0.4017],
       [-0.4702,  0.    , -0.524 , -0.4041, -0.6209, -0.4507, -0.4169,
        -0.5277, -0.6986, -0.4447],
       [-0.431 , -0.5612,  0.    , -0.4562, -0.6477, -0.4618, -0.4424,
        -0.4921, -0.6605, -0.5128],
       [-0.3848, -0.5061, -0.5229,  0.    , -0.5662, -0.462 , -0.4327,
        -0.4505, -0.5573, -0.5492],
       [-0.4427, -0.637 , -0.6228, -0.4826,  0.    , -0.4499, -0.4244,
        -0.5983, -0.764 , -0.5202],
       [-0.4221, -0.4971, -0.4706, -0.4059, -0.4775,  0.    , -0.4698,
        -0.4356, -0.4817, -0.4356],
       [-0.3865, -0.4757, -0.4661, -0.3897, -0.4669, -0.4804,  0.    ,
        -0.4492, -0.4723, -0.4263],
       [-0.4424, -0.5691, -0.4915, -0.388 , -0.615 , -0.4214, -0.4231,
         0.    , -0.6256, -0.4629],
       [-0.4519, -0.7165, -0.6481, -0.4827, -0.7668, -0.4563, -0.4331,
        -0.6107,  0.    , -0.5223],
       [-0.4188, -0.501 , -0.5367, -0